In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv("data/hius.csv")
df_2 = pd.read_csv("data/quickcharts_hius.csv")

df = df.drop(0, axis=0)
df.head()


    

#the first row is the description of the column so we should drop that as well

,GHO (CODE),GHO (DISPLAY),GHO (URL),DATASOURCE (CODE),DATASOURCE (DISPLAY),DATASOURCE (URL),PUBLISHSTATE (CODE),PUBLISHSTATE (DISPLAY),PUBLISHSTATE (URL),YEAR (CODE),...,SEX (CODE),SEX (DISPLAY),SEX (URL),Display Value,Numeric,Low,High,StdErr,StdDev,Comments
1,CHOLERA_0000000001,Number of reported cases of cholera,http://apps.who.int/gho/indicatorregistry/App_...,NaN,NaN,NaN,PUBLISHED,Published,NaN,1987,...,NaN,NaN,NaN,6,NaN,NaN,NaN,NaN,NaN,NaN
2,CHOLERA_0000000001,Number of reported cases of cholera,http://apps.who.int/gho/indicatorregistry/App_...,NaN,NaN,NaN,PUBLISHED,Published,NaN,1988,...,NaN,NaN,NaN,9,NaN,NaN,NaN,NaN,NaN,NaN
3,CHOLERA_0000000001,Number of reported cases of cholera,http://apps.who.int/gho/indicatorregistry/App_...,NaN,NaN,NaN,PUBLISHED,Published,NaN,1990,...,NaN,NaN,NaN,7,NaN,NaN,NaN,NaN,NaN,NaN
4,CHOLERA_0000000001,Number of reported cases of cholera,http://apps.who.int/gho/indicatorregistry/App_...,NaN,NaN,NaN,PUBLISHED,Published,NaN,1991,...,NaN,NaN,NaN,26,NaN,NaN,NaN,NaN,NaN,NaN
5,CHOLERA_0000000001,Number of reported cases of cholera,http://apps.who.int/gho/indicatorregistry/App_...,NaN,NaN,NaN,PUBLISHED,Published,NaN,1992,...,NaN,NaN,NaN,103,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
df_2 = df_2.drop(0, axis=0)
df_2.head()

,GHO (CODE),GHO (DISPLAY),GHO (URL),PUBLISHSTATE (CODE),PUBLISHSTATE (DISPLAY),PUBLISHSTATE (URL),YEAR (CODE),YEAR (DISPLAY),YEAR (URL),REGION (CODE),...,SEX (CODE),SEX (DISPLAY),SEX (URL),Display Value,Numeric,Low,High,StdErr,StdDev,Comments
1,MDG_0000000001,Infant mortality rate (probability of dying be...,http://apps.who.int/gho/indicatorregistry/App_...,PUBLISHED,Published,NaN,1990.0,1990,NaN,AMR,...,BTSX,Both sexes,NaN,9.4,9.40000,NaN,NaN,NaN,NaN,NaN
2,MDG_0000000001,Infant mortality rate (probability of dying be...,http://apps.who.int/gho/indicatorregistry/App_...,PUBLISHED,Published,NaN,1991.0,1991,NaN,AMR,...,BTSX,Both sexes,NaN,9.1,9.10000,NaN,NaN,NaN,NaN,NaN
3,MDG_0000000001,Infant mortality rate (probability of dying be...,http://apps.who.int/gho/indicatorregistry/App_...,PUBLISHED,Published,NaN,1992.0,1992,NaN,AMR,...,BTSX,Both sexes,NaN,8.8,8.80000,NaN,NaN,NaN,NaN,NaN
4,MDG_0000000001,Infant mortality rate (probability of dying be...,http://apps.who.int/gho/indicatorregistry/App_...,PUBLISHED,Published,NaN,1993.0,1993,NaN,AMR,...,BTSX,Both sexes,NaN,8.5,8.50000,NaN,NaN,NaN,NaN,NaN
5,MDG_0000000001,Infant mortality rate (probability of dying be...,http://apps.who.int/gho/indicatorregistry/App_...,PUBLISHED,Published,NaN,1994.0,1994,NaN,AMR,...,BTSX,Both sexes,NaN,8.2,8.20000,NaN,NaN,NaN,NaN,NaN


In [ ]:
# the difference between the two datasets
list(set(df.columns) - set(df_2.columns))

In [4]:
for column in df.columns:
    print("{} has values:\n{}\n".format(column, df[column].unique()))
    print("-"*100)

GHO (CODE) has values:
['CHOLERA_0000000001' 'CHOLERA_0000000002' 'CHOLERA_0000000003' 'CM_01'
 'CM_02' 'CM_03' 'DEVICES22' 'HIV_0000000020' 'MDG_0000000001'
 'MDG_0000000003' 'MDG_0000000005' 'MDG_0000000006' 'MDG_0000000007'
 'MDG_0000000017' 'MDG_0000000020' 'MDG_0000000023' 'MDG_0000000025'
 'MDG_0000000026' 'MDG_0000000027' 'MDG_0000000029' 'MDG_0000000031'
 'MDG_0000000033' 'MH_17' 'MH_18' 'MH_6' 'NUTRITION_564' 'SA_0000001688'
 'TB_1' 'TB_tot_newrel' 'TOBACCO_0000000192' 'WHOSIS_000001'
 'WHOSIS_000003' 'WHOSIS_000005' 'WHOSIS_000009' 'WHOSIS_000015' 'WHS10_1'
 'WHS10_8' 'WHS10_9' 'WHS2_138' 'WHS2_164' 'WHS2_165' 'WHS2_166'
 'WHS2_167' 'WHS2_168' 'WHS2_170' 'WHS2_171' 'WHS2_174' 'WHS2_514'
 'WHS2_515' 'WHS2_516' 'WHS2_523' 'WHS3_40' 'WHS3_41' 'WHS3_43' 'WHS3_45'
 'WHS3_46' 'WHS3_49' 'WHS3_50' 'WHS3_52' 'WHS3_53' 'WHS3_55' 'WHS3_56'
 'WHS3_57' 'WHS3_62' 'WHS4_100' 'WHS4_115' 'WHS4_117' 'WHS4_129'
 'WHS4_154' 'WHS6_102' 'WHS6_123' 'WHS6_125' 'WHS6_127' 'WHS6_144'
 'WHS6_148' 'WHS6

In [ ]:
#figuring out how many nans per column

# Lets count the amount of Nan



In [ ]:


#separating diseases

disList = df['GHO (CODE)'].unique()

diseases = []
for i in disList:
    tempRows = df['GHO (CODE)'] == i
    tempDis = df[tempRows]
    diseases.append(tempDis)
    
#nan values for each disease df
print('lengths are ', len(diseases), ' ', len(disList))

# for i in diseases[1]:
    
#     x = i.values[:, :-1]

#     nan_count = i.isna().sum()
#     nan_percentage = [count / x.shape[0] for count in nan_count]
#     columns = i.columns

#     threshold = 0.6

#     # And let's plot those
#     fig1 = plt.figure(figsize=(30,15))
#     ax = fig1.add_subplot(111)
#     plt.axhline(y=threshold,linewidth=1, color="k")
#     ax.bar(columns, nan_percentage)
#     plt.xticks(rotation=90)
#     ax.set_xlabel('Column')
#     ax.set_ylabel('NaN Count')
#     ax.set_title('Columns with missing values')
    

### Conclusion:

The set is as good as complete, but some columns only apply to some diseases/conditions
So, we will remove all unimportant columns for each disease.

In [ ]:
for i in diseases:
    x = i.values[:, :-1]

    nan_count = i.isna().sum()
    nan_percentage = [count / x.shape[0] for count in nan_count]
    columns = i.columns

    threshold = 0.6

    too_much_cols = []
    for j in range(len(columns)):
        if nan_percentage[j] >= threshold:
            too_much_cols.append(columns[j])
    i.drop(too_much_cols, axis=1, inplace=True)
    
diseases[0].head()

In [ ]:

print(diseases[0].dtypes)




All values are objects, change to numeric values.

In [ ]:

for i in diseases:
    for j in i['YEAR (CODE)']:
#         print(j, ' ', type(j))
#         if not int(j):
            print('calling')
#             strAr = j.split('-')
#             a = int(strAr[0])
#             b = int(strAr[1])
#             c = int((b-a)/2)
#             j = c
#         else:
#             j = int(j)
# sns.lineplot(x='YEAR (CODE)', y='Display Value', data=diseases[0])